In [4]:
import pandas as pd


In [5]:

df = pd.read_csv('data/Taylor_Swift_Genius/data.csv')
formatted_lyrics = []
for index, row in df.iterrows():
    title_entry = f"{row['Song Name']} - {row['Album']}:\n"
    lyrics_entry = f"{row['Lyrics']}\n\n"  # Add two newlines for spacing between songs
    formatted_lyrics.append(title_entry + lyrics_entry)

all_lyrics = ''.join(formatted_lyrics)
with open('lyrics_w_album.txt', 'w', encoding='utf-8') as file:
    file.write(all_lyrics)


In [7]:
df = pd.read_csv('data/Taylor_Swift_Genius/data.csv')
lyrics = df['Lyrics'].tolist()
all_lyrics = '\n\n'.join(lyrics)
with open('taylor_swift_lyrics.txt', 'w', encoding='utf-8') as file:
    file.write(all_lyrics)


In [9]:
with open('data/taylor_swift_lyrics.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [23]:
print('length of dataset in char ', len(text))
print(text[:100])

length of dataset in char  272451
She said I was seven and you were nine I looked at you like the stars that shined In the sky, the pr


In [41]:
chars = sorted(list(set(text)))
# vocab_size = len(chars)
print(''.join(chars))
# print(vocab_size)


 !"'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYabcdefghijklmnopqrstuvwxyzéíóе ​–—‘’“ 


In [43]:
# Integer to String mapping
itos = {i: ch for i, ch in enumerate(chars)}

# String to Integer mapping, the reverse of itos
stoi = {ch: i for i, ch in enumerate(chars)}

# Encoder: Convert string to list of integers
encode = lambda s: [stoi[c] for c in s]

# Decoder: Convert list of integers back to string
decode = lambda l: ''.join([itos[i] for i in l])

# Testing the functions
print(encode("hii there"))
print(decode(encode("hii there")))

[55, 56, 56, 1, 67, 55, 52, 65, 52]
hii there


In [46]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([272451]) torch.int64
tensor([41, 55, 52,  1, 66, 48, 56, 51,  1, 31])


In [47]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [51]:
block_size = 8
train_data[:block_size+1]

tensor([41, 55, 52,  1, 66, 48, 56, 51,  1])

In [52]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([41]) the target: 55
when input is tensor([41, 55]) the target: 52
when input is tensor([41, 55, 52]) the target: 1
when input is tensor([41, 55, 52,  1]) the target: 66
when input is tensor([41, 55, 52,  1, 66]) the target: 48
when input is tensor([41, 55, 52,  1, 66, 48]) the target: 56
when input is tensor([41, 55, 52,  1, 66, 48, 56]) the target: 51
when input is tensor([41, 55, 52,  1, 66, 48, 56, 51]) the target: 1


In [59]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[52, 53, 67,  1, 25, 62, 65, 61],
        [ 1, 56, 66,  1, 48, 54, 59, 62],
        [69, 52,  1, 60, 52,  7,  1, 60],
        [ 6,  1, 24, 48, 50, 58,  1, 48]])
targets:
torch.Size([4, 8])
tensor([[53, 67,  1, 25, 62, 65, 61, 52],
        [56, 66,  1, 48, 54, 59, 62, 70],
        [52,  1, 60, 52,  7,  1, 60, 72],
        [ 1, 24, 48, 50, 58,  1, 48, 61]])
----
when input is [52] the target: 53
when input is [52, 53] the target: 67
when input is [52, 53, 67] the target: 1
when input is [52, 53, 67, 1] the target: 25
when input is [52, 53, 67, 1, 25] the target: 62
when input is [52, 53, 67, 1, 25, 62] the target: 65
when input is [52, 53, 67, 1, 25, 62, 65] the target: 61
when input is [52, 53, 67, 1, 25, 62, 65, 61] the target: 52
when input is [1] the target: 56
when input is [1, 56] the target: 66
when input is [1, 56, 66] the target: 1
when input is [1, 56, 66, 1] the target: 48
when input is [1, 56, 66, 1, 48] the target: 54
when input is [1, 56, 

In [57]:
print(xb) # our input to the transformer

tensor([[ 1, 67, 56, 60, 52,  1, 47, 52],
        [52,  1, 42, 55, 56, 61, 58,  1],
        [62, 51, 72,  1, 67, 52, 59, 59],
        [50, 62, 68, 59, 51,  1, 53, 52]])


In [60]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch,Time,Channel)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 86])
tensor(5.0781, grad_fn=<NllLossBackward0>)

ICA;26​YG17Ixqíse3U0DHb5lI0‘c(W‘eNp)du5lwz1HDékvlX"'NmAl(w;0Dm​oKVKóW896’ 4m0H‘hsE?)h"oéGvB2mDbPpC40


In [61]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [107]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.33646559715271


In [113]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))




Fo Caith-at't pemyoowak tous I w, we Vr' benefofofigon Thiknau ou therou, teelldof g, gou's w y athohe tha wis be ca t me Sous t Itth) smm Ithere'sligha-owar or hacer thinoge f is Anllemanousthow  Dounonomitou Oheandyopp ave d teaul Wemsancoo rsharet foug ce o tokede we ownntivere I y my yoyo We othans w k yo a Yoourst s t Acaveshowannd kngotould lkne stethat d theve Ald wnd f But't I mayo y, mave de t thawsous f e artst tandoulis Ith, unt at w y who I ikyof st ht Cassais st mayomitrianea wike
